[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/philogicae/ai-notebooks-colab/blob/master/stable_diffusion_webui.ipynb)

# **Stable Diffusion WebUI Colab by [Philogicae](https://github.com/philogicae)**

- This notebook takes around **5min** to launch **Stable Diffusion** and generate a **public link** using Gradio (usable as webUI or to paste into the Photoshop extension).
- At launch, it will ask you for permission to connect to Google Drive (to load/save your config, output images and access your downloaded models).
- Feel free to add/remove your own **extensions**, **scripts** or **embeddings** below (Many on [Github](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Extensions) & [HuggingFace](https://huggingface.co/)).
- You can activate/deactivate the default **models** you want to load at startup, or use the [civitai](https://civitai.com/) extension,
but I strongly advise you to download your favorite models (using **Tools**) to keep them into your **Google Drive folder**: *stable-diffusion/models*
- To learn how to use the webUI -> Youtube/Discord
- To use it inside **Photoshop**, download and install the .ccx: [stable.art](https://github.com/isekaidev/stable.art#how-to-install)
- If you use a cracked version of Ps, to install the .ccx you will need: [UXP Developer Tools](https://developer.adobe.com/photoshop/uxp/devtool/installation/)
- For questions/bugs, drop me a message on [Twitter](https://twitter.com/philogicae)/[Github](https://github.com/philogicae)/[Telegram](https://t.me/philogicae).

# Tools

In [ ]:
#@markdown **Run this block to download a model from an URL directly to your Google drive:**
import os
from google.colab import drive # type: ignore
drive_path = "/content/drive"
data_dir = "/content/drive/MyDrive/stable-diffusion"

# Mount Google Drive
if not os.path.exists(drive_path):
    try:
        drive.mount(drive_path, force_remount=True)
        os.makedirs(data_dir, exist_ok=True)
    except:
        raise("Error mounting drive or with path variables")

# Download models directly into your Google Drive folder: stable-diffusion/models
models_path = f"{data_dir}/models"
os.makedirs(models_path, exist_ok=True)
%cd {models_path}

model_url = "" #@param {type:"string"}
if model_url:
    print(f"Downloading model from {model_url}")
    !wget --content-disposition {model_url}
    print(f"Downloaded model to {models_path}")
else:
    print("Error: Invalid infos")


# Setup & Run

In [ ]:
#@markdown **Drive settings**
import os
from google.colab import drive # type: ignore
drive_path = "/content/drive"
data_dir = "/content/drive/MyDrive/stable-diffusion" #@param {type:"string"}
sd_dir = "/content/stable-diffusion-webui"
%cd "/content"

### Mount Google Drive
if not os.path.exists(drive_path):
    try:
        drive.mount(drive_path, force_remount=True)
        os.makedirs(data_dir, exist_ok=True)
    except:
        raise("Error mounting drive or with path variables")

### Requirements
%env TF_CPP_MIN_LOG_LEVEL=1
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb
!apt -y install -qq aria2
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui

#@markdown **Extensions**
%cd {sd_dir}/extensions
!git clone https://github.com/etherealxx/batchlinks-webui
catppuccin_webui = True #@param {type:"boolean"}
if catppuccin_webui:
    !git clone https://github.com/catppuccin/stable-diffusion-webui {sd_dir}/extensions/stable-diffusion-webui-catppuccin
images_browser = True #@param {type:"boolean"}
if images_browser:
    !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser
stealth_pnginfo = True #@param {type:"boolean"}
if stealth_pnginfo:
    !git clone https://github.com/camenduru/sd_webui_stealth_pnginfo
civitai_browser = True #@param {type:"boolean"}
if civitai_browser:
    !git clone https://github.com/camenduru/sd-civitai-browser
deforum = True #@param {type:"boolean"}
if deforum:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui
background_remover = True #@param {type:"boolean"}
if background_remover:
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg
latent_couple = True #@param {type:"boolean"}
if latent_couple:
    !git clone https://github.com/ashen-sensored/stable-diffusion-webui-two-shot
prompt_generator = True #@param {type:"boolean"}
if prompt_generator:
    !git clone https://github.com/imrayya/stable-diffusion-webui-Prompt_Generator
openOutpaint = False #@param {type:"boolean"}
if openOutpaint:
    !git clone https://github.com/zero01101/openOutpaint-webUI-extension

#@markdown **Scripts**
%cd {sd_dir}/scripts
run_n_times = True #@param {type:"boolean"}
if run_n_times:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py

#@markdown **Embeddings**
%cd {sd_dir}/embeddings
dblx = True #@param {type:"boolean"}
if dblx:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt
negative =True #@param {type:"boolean"}
if negative:
    !git clone https://huggingface.co/embed/negative {sd_dir}/embeddings/negative

### Outputs / Config -> Saved to Google Drive
%cd {sd_dir}
output_path = f"{data_dir}/outputs"
os.makedirs(output_path, exist_ok=True)
!rm -Rf outputs && ln -s {output_path} outputs

config_path = f"{data_dir}/config"
os.makedirs(config_path, exist_ok=True)
for filename in ["config.json", "ui-config.json", "styles.csv", "artists.csv"]:
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            !touch {config_path}/{filename}
    !rm -f {filename} && ln -s {config_path}/{filename} {filename}

#@markdown **Default Models**
%cd {sd_dir}/models/Stable-diffusion
stable_diffusion_2_1_768x768 = True #@param {type:"boolean"}
if stable_diffusion_2_1_768x768:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/stable-diffusion-2-1/raw/main/v2-1_768-ema-pruned.yaml
stable_diffusion_inpainting_512x512 = False #@param {type:"boolean"}
if stable_diffusion_inpainting_512x512:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/resolve/main/512-inpainting-ema.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inpainting-inference.yaml -o 512-inpainting-ema.yaml
stable_diffusion_upscaler_512x512 = False #@param {type:"boolean"}
if stable_diffusion_upscaler_512x512:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler/resolve/main/x4-upscaler-ema.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/x4-upscaling.yaml -o x4-upscaler-ema.yaml
instruct_pix2pix_512x512 = False #@param {type:"boolean"}
if instruct_pix2pix_512x512:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.safetensors
stable_diffusion_1_5_512x512 = False #@param {type:"boolean"}
if stable_diffusion_1_5_512x512:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/v1-inference.yaml -o v1-5-pruned-emaonly.yaml
upscaler_ultrasharp = True #@param {type:"boolean"}
if upscaler_ultrasharp:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {sd_dir}/models/ESRGAN -o 4x-UltraSharp.pth

### Load Saved Models (From GoogleDrive/stable-diffusion/models)
models_path = f"{data_dir}/models"
os.makedirs(models_path, exist_ok=True)
for filename in os.listdir(models_path):
    !ln -s {models_path}/{filename} {filename}

#@markdown **[ControlNet](https://stable-diffusion-art.com/controlnet/) (To use with SD v1.5 models)**
canny = False #@param {type:"boolean"}
depth = False #@param {type:"boolean"}
hed = False #@param {type:"boolean"}
mlsd = False #@param {type:"boolean"}
normal = False #@param {type:"boolean"}
openpose = False #@param {type:"boolean"}
scribble = False #@param {type:"boolean"}
seg = False #@param {type:"boolean"}
if canny or depth or hed or mlsd or normal or openpose or scribble or seg:
    %cd {sd_dir}/extensions
    !git clone https://github.com/Mikubill/sd-webui-controlnet
    !git clone https://github.com/fkunn1326/openpose-editor
    !git clone https://github.com/hnmr293/posex
    !git clone https://github.com/jexom/sd-webui-depth-lib
    %cd {sd_dir}/extensions/sd-webui-controlnet/models
    if canny:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors
    if depth:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors
    if hed:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors
    if mlsd:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors
    if normal:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors
    if openpose:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors
    if scribble:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors
    if seg:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors

#@markdown **[ModelScope](https://github.com/deforum-art/sd-webui-text2video) (To use with SD v1.5 models)**
text2video = False #@param {type:"boolean"}
if text2video:
    %cd {sd_dir}/extensions
    !git clone https://github.com/deforum-art/sd-webui-text2video
    t2v_path = f"{sd_dir}/models/ModelScope/t2v"
    !mkdir -p {t2v_path}
    %cd {t2v_path}
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/resolve/main/VQGAN_autoencoder.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/raw/main/configuration.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/resolve/main/open_clip_pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/resolve/main/text2video_pytorch_model.pth

### Launch
%cd {sd_dir}
!git reset --hard
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' {sd_dir}/modules/shared.py
!python launch.py --theme dark --listen --xformers --api --enable-insecure-extension-access --gradio-queue --multiple